In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
# from langchain_google_genai import ChatGoogleGenerativeAI
# chat = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=api_key, temperature=0.2)
# message = "write a poem on my love for dosa"
# answer = chat.invoke("write a poem on my love for dosa")

In [3]:
from langchain_google_genai import GoogleGenerativeAI
chat = GoogleGenerativeAI(model="gemini-pro", temperature=0.2)

D:\Program Files\python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# import pprint as pp
# response = chat.invoke(
#         "What are some of the pros and cons of Python as a programming language?"
#     )
# pp.pprint(response)

In [3]:
import sqlite3
conn = sqlite3.connect('result/data.sqlite')

In [9]:
import pandas as pd
from create_table_query import CREATE_TABLE
# create the cursor object 
cur = conn.cursor() 
  
# # execute the script by creating the table 
# # named geeks1 and insert the data 
# cur.executescript(CREATE_TABLE)
# fetch all the data 
cur.execute("SELECT * from discounts")
rows = cur.fetchall()
pd.DataFrame(rows)

,0,1,2
0,1,1,10.00
1,2,2,15.00
2,3,3,20.00
3,4,4,5.00
4,5,5,25.00
5,6,6,10.00
6,7,7,30.00
7,8,8,35.00
8,9,9,40.00
9,10,10,45.00


In [7]:
import pymysql
import pandas as pd

SERVERNAME = "localhost"
DATABASE = "atliq_tshirts"
USER = "root"
PASSWORD = "bhavdeep123"

# Connect to the database
connection = pymysql.connect(host='localhost',
                             user=USER,
                             password=PASSWORD,
                             database=DATABASE,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

def preprocess_query(query):
    query = query.replace("```", "")
    query = query.replace("sql", "")
    query = query.replace("\n", " ")
    return query

def query_result(connection, query):
    with connection.cursor() as cursor:
            # Read a single record
            # sql = "SELECT * FROM t_shirts"
            cursor.execute(query)

            # Connection is not autocommit by default, so we must commit to save changes
            connection.commit()
            # Fetch all the records from SQL query output
            results = cursor.fetchall()
            # Convert results into pandas dataframe
            df = pd.DataFrame(results)
            # print(f'Successfully retrieved records')
            return df

RuntimeError: 'cryptography' package is required for sha256_password or caching_sha2_password auth methods

In [3]:
from langchain_community.utilities.sql_database import SQLDatabase
db_user = "root"
db_password = "bhavdeep123"
db_host = "localhost"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [4]:
# from langchain_experimental.sql import SQLDatabaseChain

# db_chain = SQLDatabaseChain.from_llm(chat, db, verbose=True)

# question = "How many t-shirts do we have left for nike in extra small size and white color?"
# qns1 = db_chain.run(question)


In [9]:
from langchain.chains import create_sql_query_chain
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_sql_query_chain(chat, db)

In [46]:
question1 = "How many t-shirts do we have left for nike in extra small size?"
query1 = chain.invoke(
    {
        "question": question1
    }
)
print(query1)
query1 = preprocess_query(query1)
print(query1)

# db.run(query1)
query_result(connection, query1)


```sql
SELECT 
  `stock_quantity`
FROM 
  `t_shirts`
WHERE 
  `brand` = 'Nike' AND `size` = 'XS';
```
 SELECT    `stock_quantity` FROM    `t_shirts` WHERE    `brand` = 'Nike' AND `size` = 'XS'; 


,stock_quantity
0,62
1,72


In [12]:
sql_code_1 = "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'"
qns1 = query_result(connection, sql_code_1)
qns1 = str(qns1.values[0][0])
qns1

'None'

In [13]:
question2 = "How much is the price of the inventory for all small size t-shirts"

query2 = chain.invoke(
    {
        "question": question2
    }
)
query2 = preprocess_query(query2)
print(query2)
query_result(connection, query2)

 SELECT SUM(`t_shirts`.`price` * `t_shirts`.`stock_quantity`) AS `total_price` FROM `t_shirts` WHERE `t_shirts`.`size` = 'S'; 


,total_price
0,16112


In [14]:
sql_code_2 = "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'"
qns2 = query_result(connection, sql_code_2)
qns2 = str(qns2.values[0][0])
qns2

'16112'

In [15]:

question3 = "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?"

query3 = chain.invoke(
    {
        "question": question3
    }
)
query3 = preprocess_query(query3)
print(query3)
query_result(connection, query3)

 SELECT SUM(`t_shirts`.`price` * (1 - `discounts`.`pct_discount` / 100)) AS `total_revenue` FROM `t_shirts` JOIN `discounts` ON `t_shirts`.`t_shirt_id` = `discounts`.`t_shirt_id` WHERE `t_shirts`.`brand` = 'Levi' LIMIT 5; 


,total_revenue
0,60.600000


In [16]:
sql_code_3 = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = query_result(connection, sql_code_3)
qns3 = str(qns3.values[0][0])
qns3

'16928.800000'

In [17]:
question4 = "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" 

query4 = chain.invoke(
    {
        "question": question4
    }
)
query4 = preprocess_query(query4)
print(query4)
query_result(connection, query4)

 SELECT SUM(`t_shirts`.`price` * `t_shirts`.`stock_quantity`) AS `total_revenue` FROM `t_shirts` WHERE `t_shirts`.`brand` = 'Levi'; 


,total_revenue
0,18179


In [18]:
# question4 = "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" 
sql_code_4 = "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'"

qns4 = query_result(connection, sql_code_4)
qns4 = str(qns4.values[0][0])
qns4

'18179'

In [19]:
question5 = "How many white color Levi's t shirts we have available?"

query5 = chain.invoke(
    {
        "question": question5
    }
)
query5 = preprocess_query(query5)
print(query5)
query_result(connection, query5)

 SELECT    COUNT(*) AS `num_white_levi_tshirts` FROM    `t_shirts` WHERE    `brand` = 'Levi'   AND `color` = 'White'; 


,num_white_levi_tshirts
0,4


In [20]:
sql_code_5 = "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'"

qns5 = query_result(connection, sql_code_5)
qns5 = str(qns5.values[0][0])
qns5

'168'

### Few Shot Prompting

##### We will use few shot learning to fix issues we have seen so far

In [21]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : f"{qns1}"},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': f"{qns2}"},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': f"{qns3}"} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : f"{qns4}"},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : f"{qns5}"
     }

]

## Creating Semantic Similarity Based example selector
#### create embedding on the few_shots
#### Store the embeddings in Chroma DB
#### Retrieve the the top most Semantically close example from the vector store

In [22]:
assert(False)

AssertionError: 

In [23]:
# from langchain.prompts import SemanticSimilarityExampleSelector
# from langchain_community.vectorstores import Chroma

# embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
to_vectorize = [" ".join(example.values()) for example in few_shots]
to_vectorize

["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query None",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query 16112",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query 16928.800000",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 

In [24]:
# from langchain_huggingface import HuggingFaceEmbeddings

# model_name = "sentence-transformers/all-mpnet-base-v2"
# model_kwargs = {'device': 'cpu'}
# encode_kwargs = {'normalize_embeddings': False}
# embeddings = HuggingFaceEmbeddings(
#     model_name=model_name,
#     model_kwargs=model_kwargs,
#     encode_kwargs=encode_kwargs
# )


# embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

In [25]:
# vectorstore = Chroma.from_texts(to_vectorize, embedding=embeddings, metadatas = few_shots)

In [26]:
# ## langchain-cohere embeddings
# from langchain_cohere import CohereEmbeddings

# embeddings_model = CohereEmbeddings()

In [27]:
# embeddings = embeddings_model.embed_documents(to_vectorize)
# len(embeddings), len(embeddings[0])

In [28]:
# from chromadb import Documents, EmbeddingFunction, Embeddings
# class CohereEmbeddingFunction(EmbeddingFunction):
#     def __call__(self, input: Documents) -> Embeddings:
#         embeddings_model = CohereEmbeddings()
#         embeddings = embeddings_model.embed_documents(Documents)
#         return embeddings

In [29]:
# client.delete_collection(name="t_shirts_sql")

In [31]:
# python can also run in-memory with no server running: chromadb.PersistentClient()

import chromadb
client = chromadb.HttpClient()

# # Add docs to the collection. Can also update and delete. Row-based API coming soon!
# collection = client.create_collection(name="t_shirts_sql")

# # Add docs to the collection. Can also update and delete. Row-based API coming soon!
# collection.add(
#     documents=to_vectorize, # we embed for you, or bring your own
#     metadatas=few_shots, # filter on arbitrary metadata!
#     ids=[f"doc{i+1}" for i in range(len(to_vectorize))], # must be unique for each doc 
# )

In [34]:
collection = client.get_or_create_collection("t_shirts_sql")

collection.upsert(
    documents=to_vectorize, # we embed for you, or bring your own
    metadatas=few_shots, # filter on arbitrary metadata!
    ids=[f"doc{i+1}" for i in range(len(to_vectorize))], # must be unique for each doc 
)

query_text = "How many white Nike tshirts are left in extra small size?"
results = collection.query(
    query_texts=[query_text],
    n_results=2,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
)  

In [35]:
results['metadatas']

[[{'Answer': 'None',
   'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
   'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
   'SQLResult': 'Result of the SQL query'},
  {'Answer': '168',
   'Question': "How many white color Levi's shirt I have?",
   'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
   'SQLResult': 'Result of the SQL query'}]]

### Setting up PromptTemplete using input variables

In [36]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [37]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)


Only use the following tables:
{table_info}

Question: {input}


In [38]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [39]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer"],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [40]:
print(example_prompt)

input_variables=['Answer', 'Question', 'SQLQuery', 'SQLResult'] template='\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}'


In [41]:
few_shots

[{'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query',
  'Answer': 'None'},
 {'Question': 'How much is the total price of the inventory for all S-size t-shirts?',
  'SQLQuery': "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
  'SQLResult': 'Result of the SQL query',
  'Answer': '16112'},
 {'Question': 'If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
  'SQLQuery': "SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n ",
  'SQLResult': 'Result of the SQL query',
  'Answer':

In [42]:
# query_text = "How many white Nike tshirts are left in extra small size?"
query_text = "total remaining levi white tshirt?"
# query_text = "How much is the price of all white color levi t shirts?"
# query_text = "How much is the price of the inventory for all small size t-shirts?"
# query_text = "How much is the price of all white color levi t shirts?"
# query_text = "If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?"
# query_text = "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?"
# query_text = "If we have to sell all the Ven Huson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?"
# query_text = "How much revenue  our store will generate by selling all Van Heuson TShirts without discount?"

results = collection.query(
    query_texts=[query_text],
    n_results=2,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
)  
results['metadatas'][0]

[{'Answer': '168',
  'Question': "How many white color Levi's shirt I have?",
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': 'None',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'}]

In [43]:
few_shot_prompt = FewShotPromptTemplate(
    examples=results['metadatas'][0],
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

new_chain = create_sql_query_chain(llm=chat, db=db, prompt=few_shot_prompt)

query = new_chain.invoke(
    {
        "question": query_text
    }
)
print(query_text)
query = preprocess_query(query)
print(query)
query_result(connection, query)

total remaining levi white tshirt?
SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White';


,SUM(`stock_quantity`)
0,168


In [44]:
new_chain.get_prompts()[0].pretty_print()

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: R